In [29]:
# utilities
import re
import numpy as np
import pandas as pd

# nltk
from nltk.stem import WordNetLemmatizer

#SpellCorrection
from spellchecker import SpellChecker

import string
import emoji
import chardet

In [30]:
DATASET_COLUMNS = ['date', 'username', 'text', 'polarity', 'emotion']

#Detect file encoding using chardet
with open('data.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Print the detected encoding
print("Detected encoding:", result['encoding'])

# Read the file using the detected encoding
df = pd.read_csv('data.csv', encoding=result['encoding'], names=DATASET_COLUMNS)
df.sample(5)

Detected encoding: UTF-8-SIG


,date,username,text,polarity,emotion
407,10/28/23,@ja1405_ja,"#Btc back to ATH is only 2X, this can happen ...",2,anticipation
563,10/30/23,@CryptoChef_M,#Bitcoin Weekly Forecast📌‼️ Oct. 30 - Nov.05 K...,2,happy
511,10/27/23,@Rao_Cash,"Rao Cash is the new bitcoin killer, earn X100,...",2,surprised
508,10/27/23,@VipinvestmentsI,The future of digital asset management is here...,2,happy
127,"8:05 PM · Oct 23, 2023",@Eldorado_krypto,📝 The ruling urges banks not to freeze account...,2,happy


In [31]:
#Data preprocessing
data=df[['text','polarity', 'emotion']]

In [32]:
data['polarity'].unique()

array([2, 1])

In [33]:
data_pos = data[data['polarity'] == 2]
data_neu = data[data['polarity'] == 1]
data_neg = data[data['polarity'] == 0]

In [34]:
data_pos = data_pos.iloc[:int(100)]
data_neu = data_neu.iloc[:int(100)]
data_neg = data_neg.iloc[:int(100)] 

In [35]:
dataset = pd.concat([data_pos, data_neu, data_neg])

In [36]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].tail()

450    Hey Websec fam, We're collaborating with @jobs...
452    Shill me your next Big #Crypto ready to Explod...
457    The eyes of the Geishas...watching you. Why ar...
461    What is the concept that a blockchain cannot a...
462    Web innovation: endless opportunities. #crypto...
Name: text, dtype: object

In [37]:
emoticons_to_keep = [
    '💰', '📈', '🤣', '🎊', '😂', '😭', '🙁', '😞', '💔', '😢', '😮', '😵', '🙀',
    '😱', '❗', '😠', '😡', '😤', '👎', '🔪', '🌕', '🚀', '💎', '👀', '💭', '📉',
    '😨', '😩', '😰', '💸'
]

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove hashtags and mentions
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove special characters except for emoticons
    text = re.sub(r'[^\w\s.!?{}]+'.format(''.join(emoticons_to_keep)), '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the modified cleaning function to the 'text' column in your dataset
dataset['text'] = dataset['text'].apply(clean_tweet)

# Display the 'text' column in the entire dataset
print(dataset['text'])

0      BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1            HAY bullflag breakout👀 Lets fill that wick🚀
2      Did you guys see how is doing a pitch with a d...
3      GN Fam going early to bed been up since or AM ...
4      You think this week has been fun?!? 😂😂😂😂 Wait ...
                             ...                        
450    Hey Websec fam Were collaborating with Get top...
452            Shill me your next Big ready to Explode🚀📈
457    The eyes of the Geishas...watching you. Why ar...
461    What is the concept that a blockchain cannot a...
462                Web innovation endless opportunities.
Name: text, Length: 200, dtype: object


In [38]:
# Initialize SpellChecker only once to avoid re-creation for each call
spell = SpellChecker()

# Function for spell correction
def spell_correction(text):
    words = text.split()
    misspelled = spell.unknown(words)
    corrected_words = []
    for word in words:
        if word in misspelled:
            corrected_word = spell.correction(word)
            # Check if the correction is not None, otherwise use the original word
            corrected_words.append(corrected_word if corrected_word is not None else word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Apply spell correction to the entire 'text' column
dataset['text'] = dataset['text'].apply(spell_correction)

# Display the entire dataset
print(dataset)

                                                  text  polarity       emotion
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...         2         happy
1            HAY bullfrog breakout Lets fill that wick         2  anticipation
2    Did you guys see how is doing a pitch with a d...         2         happy
3    GN Fam going early to bed been up since or AM ...         2         happy
4    You think this week has been fun?!? 😂😂😂😂 Wait ...         2  anticipation
..                                                 ...       ...           ...
450  Hey Websec fam Were collaborating with Get top...         1       Neutral
452          Shill me your next Big ready to Explode🚀📈         1       Neutral
457  The eyes of the Geishas...watching you Why are...         1       Neutral
461  What is the concept that a blockchain cannot a...         1       Neutral
462               Web innovation endless opportunities         1       Neutral

[200 rows x 3 columns]


In [39]:
#Define the emoticon dictionary outside the function for a wider scope
emoticon_dict = {
    ":)": "smile ",
    ":(": "sad ",
    ":D": "laugh ",
    "😊": "smiling face with smiling eyes ",
    "😃": "grinning face with big eyes ",
    "😉": "winking face ",
    "👌": "OK hand ",
    "👍": "Thumbs up ",
    "😁": "beaming face with smiling eyes ",
    "😂": "face with tears of joy ",
    "😄": "grinning face with smiling eyes ",
    "😅": "grinning face with sweat ",
    "😆": "grinning squinting face ",
    "😇": "smiling face with halo ",
    "😞": "disappointed face ",
    "😔": "pensive face ",
    "😑": "expressionless face ",
    "😒": "unamused face ",
    "😓": "downcast face with sweat ",
    "😕": "confused face ",
    "😖": "confounded face ",
    "💰": "Money Bag ",
    "📈": "Up Trend ",
    "🤣": "Rolling on the Floor Laughing ",
    "🎊": "Confetti Ball ",
    "😭": "Loudly Crying ",
    "🙁": "Slightly frowning face ",
    "💔": "Broken Heart ",
    "😢": "Crying Face ",
    "😮": "Face with Open Mouth ",
    "😵": "Dizzy Face ",
    "🙀": "Weary Cat ",
    "😱": "Face Screaming in Fear ",
    "❗": "Exclamation Mark ",
    "😠": "Angry Face ",
    "😡": "Pouting Face ",
    "😤": "Face with Steam from Nose ",
    "👎": "Thumbs Down ",
    "🔪": "Hocho ",
    "🌕": "Moon ",
    "🚀": "Rocket ",
    "💎": "Diamond ",
    "👀": "Eyes ",
    "💭": "Thought Balloon ",
    "📉": "Down Trend ",
    "😨": "Fearful Face ",
    "😩": "Weary Face ",
    "😰": "Anxious Face with Fear ",
    "💸": "Money with Wings "
}

# Emoticon to word conversion function
def convert_emoticons_to_words(text):
    changed_emoticons = 0  # Variable to count the number of changed emoticons
    for emoticon, word in emoticon_dict.items():
        while emoticon in text:
            text = text.replace(emoticon, word + " ", 1)
            changed_emoticons += 1
    return text, changed_emoticons

# Apply the function and count emoticons for each row
def apply_conversion(text):
    converted_text, count = convert_emoticons_to_words(text)
    return pd.Series([converted_text, count], index=['converted_text', 'emoticons_count'])

conversion_results = dataset['text'].apply(apply_conversion)
dataset['converted_text'] = conversion_results['converted_text']
dataset['emoticons_count'] = conversion_results['emoticons_count']
print("Emoticons converted to words in 'converted_text' column.")
print(dataset[['converted_text', 'emoticons_count']].head())

Emoticons converted to words in 'converted_text' column.
                                      converted_text  emoticons_count
0  BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...                0
1          HAY bullfrog breakout Lets fill that wick                0
2  Did you guys see how is doing a pitch with a d...                0
3  GN Fam going early to bed been up since or AM ...                0
4  You think this week has been fun?!? face with ...               13


In [40]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [41]:
# Stopwords removal applied separately after the option has been chosen and processed
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Apply the stopwords cleaning after the loop, once the 'text' column has been updated accordingly
dataset['text'] = dataset['converted_text'].apply(cleaning_stopwords)
print("Stopwords removed from 'text' column.")
print(dataset['text'].head())

Stopwords removed from 'text' column.
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1                 HAY bullfrog breakout Lets fill wick
2    Did guys see pitch deck reaching community Tha...
3    GN Fam going early bed since AM morning nonsto...
4    You think week fun?!? face tears joy face tear...
Name: text, dtype: object


In [42]:
# Function to clean repeating words
def cleaning_repeating_words(text):
    # This regex pattern targets whole words that are repeated
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

# Assuming 'dataset' is a pandas DataFrame and 'text' is a column in it
# Apply the cleaning function for repeating words to each row in the 'text' column
dataset['text'] = dataset['text'].apply(cleaning_repeating_words)
print("Repeating words cleaned from 'text' column.")
print(dataset['text'].head())

Repeating words cleaned from 'text' column.
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1                 HAY bullfrog breakout Lets fill wick
2    Did guys see pitch deck reaching community Tha...
3    GN Fam going early bed since AM morning nonsto...
4    You think week fun?!? face tears joy face tear...
Name: text, dtype: object


In [43]:
dataset['text']=dataset['text'].str.lower()
dataset['text'].head()

0    btc on glp resistance for now play safe if u r...
1                 hay bullfrog breakout lets fill wick
2    did guys see pitch deck reaching community tha...
3    gn fam going early bed since am morning nonsto...
4    you think week fun?!? face tears joy face tear...
Name: text, dtype: object

In [44]:
from nltk.tokenize import RegexpTokenizer

# The pattern matches word characters (\w) and punctuation marks ([^\w\s])
tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')

# Applying the modified tokenizer to the dataset
dataset['text'] = dataset['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [45]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['text']= dataset['text'].apply(lambda x: stemming_on_text(x))
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [46]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
dataset['text'] = dataset['text'].apply(lambda x: lemmatizer_on_text(x))
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [47]:
# All necessary imports should be placed at the top of the script
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import numpy as np


# Ensure `dataset` is correctly loaded with 'text' column
# dataset = pd.read_csv('your_dataset.csv') # Uncomment and set your dataset path
# Make sure 'text', 'polarity', and 'emotion' columns exist
# print(dataset.columns) # Uncomment to check columns

# Initialize the tokenizer with your dataset
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['text'])

# Function to create LSTM model, now with optimizer initialization inside
def create_lstm_model(input_length, num_classes):
    # Initialize the optimizer with the desired settings inside the function
    optimizer = Adam(learning_rate=0.0001)
    
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_length))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # The optimizer is used here directly after its initialization
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Adjust the learning rate
optimizer = Adam(learning_rate=0.0001)

# Prepare the dataset for training
sequences = tokenizer.texts_to_sequences(dataset['text'])
X = pad_sequences(sequences, maxlen=50)

# Convert labels to one-hot encoding
encoder_polarity = LabelEncoder()
y_polarity = to_categorical(encoder_polarity.fit_transform(dataset['polarity']))

encoder_emotion = LabelEncoder()
y_emotion = to_categorical(encoder_emotion.fit_transform(dataset['emotion']))

# Split the data into training and testing sets
X_train_polarity, X_test_polarity, y_train_polarity, y_test_polarity = train_test_split(X, y_polarity, test_size=0.2, random_state=42)
X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion = train_test_split(X, y_emotion, test_size=0.2, random_state=42)

# When creating the models, we do not pass the optimizer as a parameter anymore
lstm_model_polarity = create_lstm_model(50, y_polarity.shape[1])
lstm_model_emotion = create_lstm_model(50, y_emotion.shape[1])

# Initialize EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the LSTM models with EarlyStopping and validation data
lstm_model_polarity.fit(
    X_train_polarity, y_train_polarity, 
    epochs=10, 
    batch_size=64,  # Ensure batch size divides the number of samples evenly or set shuffle=True
    validation_data=(X_test_polarity, y_test_polarity),
    callbacks=[early_stopping]
)
lstm_model_emotion.fit(
    X_train_emotion, y_train_emotion, 
    epochs=10, 
    batch_size=64,  # Same batch size for consistency and ease of computation
    validation_data=(X_test_emotion, y_test_emotion),
    callbacks=[early_stopping]
)

# Save the trained LSTM models
lstm_model_polarity.save('lstm_polarity_model.h5')
lstm_model_emotion.save('lstm_emotion_model.h5')


Epoch 1/10
3/3 [==============================] - 4s 418ms/step - loss: 0.6918 - accuracy: 0.5562 - val_loss: 0.6923 - val_accuracy: 0.4750
Epoch 2/10
3/3 [==============================] - 0s 95ms/step - loss: 0.6915 - accuracy: 0.5437 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 86ms/step - loss: 0.6904 - accuracy: 0.5688 - val_loss: 0.6912 - val_accuracy: 0.5250
Epoch 4/10
3/3 [==============================] - 0s 88ms/step - loss: 0.6888 - accuracy: 0.6125 - val_loss: 0.6907 - val_accuracy: 0.5500
Epoch 5/10
3/3 [==============================] - 0s 86ms/step - loss: 0.6885 - accuracy: 0.6250 - val_loss: 0.6902 - val_accuracy: 0.6000
Epoch 6/10
3/3 [==============================] - 0s 96ms/step - loss: 0.6869 - accuracy: 0.6500 - val_loss: 0.6897 - val_accuracy: 0.6750
Epoch 7/10
3/3 [==============================] - 0s 95ms/step - loss: 0.6845 - accuracy: 0.7688 - val_loss: 0.6891 - val_accuracy: 0.6500
Epoch 8/10
3/3 [==========

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:

def extract_features(model, data):
    # Create a model that will return the outputs of the LSTM layer
    # Adjust the layer index (-2) based on where your LSTM layer is located within the model
    intermediate_layer_model = Model(inputs=model.input, 
                                     outputs=model.layers[-2].output)
    # Get the features from the data using the intermediate model
    features = intermediate_layer_model.predict(data)
    return features

# Extract features with LSTM for SVM training
# Assuming lstm_model_polarity and lstm_model_emotion are already defined and trained LSTM models
X_train_features_polarity = extract_features(lstm_model_polarity, X_train_polarity)
X_train_features_emotion = extract_features(lstm_model_emotion, X_train_emotion)


# Train SVM for polarity and emotion
svm_classifier_polarity = SVC(kernel='linear', probability=True)
svm_classifier_emotion = SVC(kernel='linear', probability=True)

svm_classifier_polarity.fit(X_train_features_polarity, np.argmax(y_train_polarity, axis=1))
svm_classifier_emotion.fit(X_train_features_emotion, np.argmax(y_train_emotion, axis=1))



5/5 [==============================] - 1s 11ms/step


SVC(kernel='linear', probability=True)

In [49]:
# Save the trained SVM models using joblib or pickle
import joblib
joblib.dump(svm_classifier_polarity, 'svm_polarity_model.joblib')
joblib.dump(svm_classifier_emotion, 'svm_emotion_model.joblib')

['svm_emotion_model.joblib']

In [50]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set for polarity
y_pred_polarity = lstm_model_polarity.predict(X_test_polarity)
# Convert predictions from one-hot encoded to label encoded for evaluation
y_pred_polarity = np.argmax(y_pred_polarity, axis=1)
# Convert ground truth from one-hot encoded to label encoded for evaluation
y_true_polarity = np.argmax(y_test_polarity, axis=1)

# Calculate precision, recall, and F-measure for polarity
precision_polarity = precision_score(y_true_polarity, y_pred_polarity, average='weighted')
recall_polarity = recall_score(y_true_polarity, y_pred_polarity, average='weighted')
f1_score_polarity = f1_score(y_true_polarity, y_pred_polarity, average='weighted')

print(f'Polarity Precision: {precision_polarity:.4f}')
print(f'Polarity Recall: {recall_polarity:.4f}')
print(f'Polarity F1 Score: {f1_score_polarity:.4f}')

2/2 [==============================] - 1s 10ms/step
Polarity Precision: 0.6262
Polarity Recall: 0.6250
Polarity F1 Score: 0.6252


In [51]:
# Make predictions on the test set for emotion
y_pred_emotion = lstm_model_emotion.predict(X_test_emotion)
# Convert predictions from one-hot encoded to label encoded for evaluation
y_pred_emotion = np.argmax(y_pred_emotion, axis=1)
# Convert ground truth from one-hot encoded to label encoded for evaluation
y_true_emotion = np.argmax(y_test_emotion, axis=1)

# Calculate precision, recall, and F-measure for emotion
precision_emotion = precision_score(y_true_emotion, y_pred_emotion, average='weighted')
recall_emotion = recall_score(y_true_emotion, y_pred_emotion, average='weighted')
f1_score_emotion = f1_score(y_true_emotion, y_pred_emotion, average='weighted')

print(f'Emotion Precision: {precision_emotion:.4f}')
print(f'Emotion Recall: {recall_emotion:.4f}')
print(f'Emotion F1 Score: {f1_score_emotion:.4f}')

2/2 [==============================] - 1s 8ms/step
Emotion Precision: 0.4598
Emotion Recall: 0.4500
Emotion F1 Score: 0.3835


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
